In [18]:
# Load data ke dalam data frame 
import pandas as pd

# Spesifikasi encoding diperlukan karena data tidak menggunakan UTF-8
df = pd.read_csv('scraping_skingame.csv', encoding='latin-1')

df.head()

,Review,Product,Rating,Username,Date
0,No review found,Skin Game Kind Watery Moisturizer - Pelembab W...,bintang 5,Budi,2024-01-27
1,Ini parah banget seller dan pengiriman kurir. ...,Skin Game Kind Watery Moisturizer - Pelembab W...,bintang 1,Glenn,2024-01-27
2,tipe kulitku sensitif,Skin Game Kind Watery Moisturizer - Pelembab W...,bintang 5,C***h,2024-01-26
3,baru pertama kali coba. Baca review banyak yan...,Skin Game Acne Warrior Gel 15 ml - Krim Jerawat,bintang 5,Retno,2024-01-25
4,kotaknya penyok. untung pake bubble wrap yg te...,Skin Game Travel Bundle,bintang 5,keza,2024-01-23


In [19]:
# Drop kolom yang tidak terpakai
df = df.drop(df.iloc[:,1:], axis=1)

# Cek data teratas
df.head()

,Review
0,No review found
1,Ini parah banget seller dan pengiriman kurir. ...
2,tipe kulitku sensitif
3,baru pertama kali coba. Baca review banyak yan...
4,kotaknya penyok. untung pake bubble wrap yg te...


PREPROCESSING

In [20]:
# PROSES CASE FOLDING (menjadikan semua huruf di kolom Review menjadi huruf kecil)
df['Review'] = df['Review'].str.lower()

print('Case Folding Result: \n')
df.head()

Case Folding Result: 



,Review
0,no review found
1,ini parah banget seller dan pengiriman kurir. ...
2,tipe kulitku sensitif
3,baru pertama kali coba. baca review banyak yan...
4,kotaknya penyok. untung pake bubble wrap yg te...


In [21]:
# import hasil case folding menjadi csv
df.to_csv("casefolding_skingame.csv")

In [22]:
# CLEANING
import re # regex library

def remove_tweet_special(text):
    # remove tab, new line, and back slice
    text = text.replace('\\t'," ").replace('\\n', " ").replace('\\u', " ").replace('\\',"")
    # remove non ASCII (emoticon, chinese word, etc)
    text = text.encode('ascii', 'replace').decode('ascii')
    # remove mention, link, hashtag
    text = ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)"," ", text).split())
    # remove incomplete URL
    return text.replace("http://", " ").replace("https://", " ")

df['Review'] = df['Review'].apply(remove_tweet_special)

# remove number
def remove_number(text):
    return  re.sub(r"\d+", "", text)

df['Review'] = df['Review'].apply(remove_number)

# remove punctuation and replace with space
def remove_punctuation(text):
    return re.sub(r'[.,]', ' ', text)

df['Review'] = df['Review'].apply(remove_punctuation)

# remove punctuation
def remove_symbol(text):
    # Menghapus simbol-simbol tidak standar dan menggantinya dengan spasi
    cleaned_text = re.sub(r'[^\w\s]', ' ', text)
    
    # Menghapus multiple whitespace
    cleaned_text = re.sub('\s+', ' ', cleaned_text).strip()
    
    return cleaned_text

df['Review'] = df['Review'].apply(remove_symbol)

# Menentukan ambang batas panjang string acak
threshold_length = 20

# Fungsi untuk menghapus string acak berdasarkan panjangnya
def hapus_string_acak_dengan_panjang(text):
    # Memeriksa panjang string dan menghapus jika melebihi ambang batas
    return ' '.join(word for word in text.split() if len(word) <= threshold_length)

# Menggunakan fungsi untuk menghapus string acak pada kolom 'Text'
df['Review'] = df['Review'].apply(hapus_string_acak_dengan_panjang)

#remove whitespace leading & trailing
def remove_whitespace_LT(text):
    return text.strip()

df['Review'] = df['Review'].apply(remove_whitespace_LT)

#remove multiple whitespace into single whitespace
def remove_whitespace_multiple(text):
    return re.sub('\s+',' ',text)

df['Review'] = df['Review'].apply(remove_whitespace_multiple)

# remove single char
def remove_single_char(text):
    return re.sub(r"\b[a-zA-Z]\b", "", text)

df['Review'] = df['Review'].apply(remove_single_char)

def remove_laughter(text):
    laughter_patterns = r'\b((ha)+h*|(he)+h*|(hi)+h*|(wk)+w*k*|(eh)+e*|(ah)+a*|(ih)+i*|(kw)+k*w*|(hem)+m*)\b'
    return re.sub(laughter_patterns, '', text, flags=re.IGNORECASE)

df['Review'] = df['Review'].apply(remove_laughter)

print('Cleaning Result : \n') 
print(df.head())

Cleaning Result : 

                                              Review
0                                    no review found
1  ini parah banget seller dan pengiriman kurir u...
2                              tipe kulitku sensitif
3  baru pertama kali coba baca review banyak yang...
4  kotaknya penyok untung pake bubble wrap yg teb...


In [23]:
# import hasil cleaning menjadi csv
df.to_csv("cleaning_skingame.csv")

In [24]:
# REMOVE DUPLICATE
df = df.drop_duplicates()
df = df.reset_index(drop=True)
# Menghapus baris yang kosong
df = df.dropna(subset=['Review'])
# Menghapus baris yang hanya berisi spasi atau whitespace
df = df[df['Review'].str.strip() != '']
df.head()

,Review
0,no review found
1,ini parah banget seller dan pengiriman kurir u...
2,tipe kulitku sensitif
3,baru pertama kali coba baca review banyak yang...
4,kotaknya penyok untung pake bubble wrap yg teb...


In [25]:
# import hasil remove duplicate menjadi csv
df.to_csv("removedup_skingame.csv")

In [26]:
# NORMALIZATION
import pandas as pd
import re

slang_dictionary = pd.read_csv('../colloquial-indonesian-lexicon2.csv')
slang_dict = pd.Series(slang_dictionary['formal'].values,index=slang_dictionary['slang']).to_dict()

slang_dictionary.head()

,slang,formal,In-dictionary,context,category1,category2,category3
0,woww,wow,1.0,wow,elongasi,0,0
1,aminn,amin,1.0,Selamat ulang tahun kakak tulus semoga panjang...,elongasi,0,0
2,met,selamat,1.0,Met hari netaas kak!? Wish you all the best @t...,abreviasi,0,0
3,netaas,menetas,1.0,Met hari netaas kak!? Wish you all the best @t...,afiksasi,elongasi,0
4,keberpa,keberapa,0.0,Birthday yg keberpa kak?,abreviasi,0,0


In [27]:
# Normalisasi kata menggunakan kamus colloquial-indonesian-lexicon2.csv
def Slangwords(text, slang_dict):
    for word in text.split():
        if word in slang_dict.keys():
            # menambahkan \b untuk menandakan batas kata di sekitar kata slang
            text = re.sub(r'\b{}\b'.format(re.escape(word)), slang_dict[word], text)
    
    text = re.sub('@[\w]+', '', text)
    return text

df['Review_norm'] = df['Review'].apply(lambda x: Slangwords(x, slang_dict))
print(df.head())

                                              Review  \
0                                    no review found   
1  ini parah banget seller dan pengiriman kurir u...   
2                              tipe kulitku sensitif   
3  baru pertama kali coba baca review banyak yang...   
4  kotaknya penyok untung pake bubble wrap yg teb...   

                                         Review_norm  
0                                    no review found  
1  ini parah banget seller dan pengiriman kurir s...  
2                              tipe kulitku sensitif  
3  baru pertama kali coba baca review banyak yang...  
4  kotaknya penyok untung pakai bubble wrap yang ...  


In [28]:
# import hasil replace slang words menjadi csv
df.to_csv("normalization_skingame.csv")

In [29]:
# PROSES TOKENIZING (word_tokenize() untuk memecah string kedalam tokens)

# import word_tokenize & FreqDist from NLTK
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist

# Tokenizing
# NLTK word tokenize 
def word_tokenize_wrapper(text):
    return word_tokenize(text)

df['review_tokens'] = df['Review_norm'].apply(word_tokenize_wrapper)

print('Tokenizing Result : \n') 
print(df.head())

Tokenizing Result : 

                                              Review  \
0                                    no review found   
1  ini parah banget seller dan pengiriman kurir u...   
2                              tipe kulitku sensitif   
3  baru pertama kali coba baca review banyak yang...   
4  kotaknya penyok untung pake bubble wrap yg teb...   

                                         Review_norm  \
0                                    no review found   
1  ini parah banget seller dan pengiriman kurir s...   
2                              tipe kulitku sensitif   
3  baru pertama kali coba baca review banyak yang...   
4  kotaknya penyok untung pakai bubble wrap yang ...   

                                       review_tokens  
0                                [no, review, found]  
1  [ini, parah, banget, seller, dan, pengiriman, ...  
2                          [tipe, kulitku, sensitif]  
3  [baru, pertama, kali, coba, baca, review, bany...  
4  [kotaknya, penyok, untung,

In [30]:
# import hasil tokenizing menjadi csv
df.to_csv("tokenizing_skingame.csv")

In [31]:
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
# PROSES FILTERING (Stopword Removal) menggunakan library sastrawi
factory = StopWordRemoverFactory()
stopword = factory.create_stop_word_remover()

# remove stopword pada list token
def stopwords_removal(words):
    filtered_words = [stopword.remove(w) for w in words]
    filtered_words = [word for word in filtered_words if word != '']  # Memfilter kata-kata kosong
    return filtered_words

df['review_tokens_SR'] = df['review_tokens'].apply(stopwords_removal) 

print(df.head())

                                              Review  \
0                                    no review found   
1  ini parah banget seller dan pengiriman kurir u...   
2                              tipe kulitku sensitif   
3  baru pertama kali coba baca review banyak yang...   
4  kotaknya penyok untung pake bubble wrap yg teb...   

                                         Review_norm  \
0                                    no review found   
1  ini parah banget seller dan pengiriman kurir s...   
2                              tipe kulitku sensitif   
3  baru pertama kali coba baca review banyak yang...   
4  kotaknya penyok untung pakai bubble wrap yang ...   

                                       review_tokens  \
0                                [no, review, found]   
1  [ini, parah, banget, seller, dan, pengiriman, ...   
2                          [tipe, kulitku, sensitif]   
3  [baru, pertama, kali, coba, baca, review, bany...   
4  [kotaknya, penyok, untung, pakai, bubble, w

In [32]:
# import hasil stopword removal menjadi csv
df.to_csv("stopword_skingame.csv")

In [33]:
# PROSES LEMMATIZATION 
# import Sastrawi package
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import swifter

# create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# stemmed
def stemmed_wrapper(term):
    return stemmer.stem(term)

term_dict = {}

for document in df['review_tokens_SR']:
    for term in document:
        if term not in term_dict:
            term_dict[term] = ' '
            
print(len(term_dict))
print("------------------------")

for term in term_dict:
    term_dict[term] = stemmed_wrapper(term)
    print(term,":" ,term_dict[term])
    
print(term_dict)
print("------------------------")


# apply stemmed term to dataframe
def get_stemmed_term(document):
    return [term_dict[term] for term in document]

df['review_tokens_stemmed'] = df['review_tokens_SR'].swifter.apply(get_stemmed_term)
print(df['review_tokens_stemmed'])
print(df.head())

1334
------------------------
no : no


review : review
found : found
parah : parah
banget : banget
seller : seller
pengiriman : kirim
kurir : kurir
lama : lama
kirimnya : kirim
plus : plus
waktu : waktu
dibuka : buka
paketnya : paket
rusak : rusak
enggak : enggak
jelas : jelas
professional : professional
niat : niat
jualan : jual
sama : sama
sekali : sekali
meskipun : meski
official : official
store : store
hargai : harga
customer : customer
membungkus : bungkus
paket : paket
baik : baik
benar : benar
tipe : tipe
kulitku : kulit
sensitif : sensitif
baru : baru
pertama : pertama
kali : kali
coba : coba
baca : baca
banyak : banyak
kasih : kasih
nilai : nilai
bagus : bagus
semoga : moga
cocok : cocok
kotaknya : kotak
penyok : penyok
untung : untung
pakai : pakai
bubble : bubble
wrap : wrap
tebal : tebal
barang : barang
aman : aman
deh : deh
for : for
the : the
first : first
time : time
mau : mau
mencoba : coba
moisturizer : moisturizer
viral : viral
bumil : bumil
enak : enak
bahannya : bahan
lebih : lebih
formula : formula
muk

Pandas Apply: 100%|██████████| 454/454 [00:00<00:00, 52739.55it/s]

0                                    [no, review, found]
1      [parah, banget, seller, kirim, kurir, lama, ba...
2                                [tipe, kulit, sensitif]
3      [baru, pertama, kali, coba, baca, review, bany...
4      [kotak, penyok, untung, pakai, bubble, wrap, t...
                             ...                        
450                                 [moga, cocok, kulit]
451                                    [tekstur, ringan]
452    [datang, cepat, aman, langsung, pakai, enak, r...
453    [cepat, datang, semua, barang, aman, enak, min...
454    [harga, affordable, promo, setia, sama, moistu...
Name: review_tokens_stemmed, Length: 454, dtype: object
                                              Review  \
0                                    no review found   
1  ini parah banget seller dan pengiriman kurir u...   
2                              tipe kulitku sensitif   
3  baru pertama kali coba baca review banyak yang...   
4  kotaknya penyok untung pake bubble

In [34]:
# Menghapus baris yang kosong
df = df.dropna(subset=['review_tokens_stemmed'])
# Menghapus baris yang hanya berisi spasi atau whitespace
df = df[df['review_tokens_stemmed'].str.strip() != '']
# Menghapus baris yang memiliki list kosong
df = df[df['review_tokens_stemmed'].apply(lambda x: x != [])]
df.head(15)

,Review,Review_norm,review_tokens,review_tokens_SR,review_tokens_stemmed
0,no review found,no review found,"[no, review, found]","[no, review, found]","[no, review, found]"
1,ini parah banget seller dan pengiriman kurir u...,ini parah banget seller dan pengiriman kurir s...,"[ini, parah, banget, seller, dan, pengiriman, ...","[parah, banget, seller, pengiriman, kurir, lam...","[parah, banget, seller, kirim, kurir, lama, ba..."
2,tipe kulitku sensitif,tipe kulitku sensitif,"[tipe, kulitku, sensitif]","[tipe, kulitku, sensitif]","[tipe, kulit, sensitif]"
3,baru pertama kali coba baca review banyak yang...,baru pertama kali coba baca review banyak yang...,"[baru, pertama, kali, coba, baca, review, bany...","[baru, pertama, kali, coba, baca, review, bany...","[baru, pertama, kali, coba, baca, review, bany..."
4,kotaknya penyok untung pake bubble wrap yg teb...,kotaknya penyok untung pakai bubble wrap yang ...,"[kotaknya, penyok, untung, pakai, bubble, wrap...","[kotaknya, penyok, untung, pakai, bubble, wrap...","[kotak, penyok, untung, pakai, bubble, wrap, t..."
5,for the first time mau cobain moisturizer vira...,for the first time mau mencoba moisturizer vir...,"[for, the, first, time, mau, mencoba, moisturi...","[for, the, first, time, mau, mencoba, moisturi...","[for, the, first, time, mau, coba, moisturizer..."
6,enak bahannya lebih cocok daripada formula lam...,enak bahannya lebih cocok daripada formula lam...,"[enak, bahannya, lebih, cocok, daripada, formu...","[enak, bahannya, lebih, cocok, formula, lama, ...","[enak, bahan, lebih, cocok, formula, lama, muk..."
7,gentle moist dan isinya banyak love the new pa...,gentle moist dan isinya banyak love the new pa...,"[gentle, moist, dan, isinya, banyak, love, the...","[gentle, moist, isinya, banyak, love, the, new...","[gentle, moist, isi, banyak, love, the, new, p..."
8,bahannya ringan lembab dan ukuran cocok untuk ...,bahannya ringan lembab dan ukuran cocok untuk ...,"[bahannya, ringan, lembab, dan, ukuran, cocok,...","[bahannya, ringan, lembab, ukuran, cocok, diba...","[bahan, ringan, lembab, ukur, cocok, bawa, tra..."
9,tipe kulitku berminyak masalah kulitku pori be...,tipe kulitku berminyak masalah kulitku pori be...,"[tipe, kulitku, berminyak, masalah, kulitku, p...","[tipe, kulitku, berminyak, masalah, kulitku, p...","[tipe, kulit, minyak, masalah, kulit, pori, be..."


In [35]:
# import hasil preprocessing menjadi csv
df.to_csv("lemma_skingame.csv")